In [6]:
import import_ipynb
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
import numpy as np
from bert.tokenization import FullTokenizer
from tqdm import tqdm_notebook
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import bertkeras as bk

In [7]:
df= pd.read_csv('data_5_products.csv', index_col='Complaint ID', usecols=['Product','Sub-product', 'Issue', 'Sub-issue', 'Consumer complaint narrative', 'Complaint ID'])

In [8]:
max_seq_length = 256

In [9]:
df = df.fillna("")
# df[df['Consumer complaint narrative'].str.len()<1000] we need to drop short texts

In [10]:
df.count()

Product                         36958
Sub-product                     36958
Issue                           36958
Sub-issue                       36958
Consumer complaint narrative    36958
dtype: int64

In [11]:
#df['Category'] = df["Product"] + " - " + df['Sub-product'] 
df.head()

,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative
Complaint ID,,,,,
1297939,Credit card,,Other,,Received Capital One charge card offer XXXX. A...
1296693,Credit card,,Rewards,,I 'm a longtime member of Charter One Bank/RBS...
1296727,Debt collection,Payday loan,Improper contact or sharing of info,Talked to a third party about my debt,I received a call from a XXXX XXXX from XXXX @...
1296785,Credit reporting,,Incorrect information on credit report,Information is not mine,I had my purse stolen in 2007. They never foun...
1295146,Debt collection,Mortgage,Communication tactics,Threatened to take legal action,"Going through a divorce, my ex and I were unab..."


In [12]:
labels = pd.get_dummies(df['Product'])
texts = df['Consumer complaint narrative']


In [33]:
txt_train, txt_test, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.50, random_state=42)
txt_test.head()

Complaint ID
1891813    I called the suntrust executive office on XXXX...
1759696    In XXXX XXXX, there were a number of unauthori...
1595043    Due to financial hardship contacted XXXX to ob...
1605621    My husband and I have a mortgage on our home t...
1746681    I got invalid interest charges every month alt...
Name: Consumer complaint narrative, dtype: object

In [36]:
len(train_labels)

18479

In [29]:
tokenizer = bk.create_tokenizer_from_hub_module()

In [35]:
# Convert data to InputExample format
train_examples = bk.convert_text_to_examples(txt_train, train_labels)
test_examples = bk.convert_text_to_examples(txt_test, test_labels)

In [31]:




# Convert to features
(train_input_ids, train_input_masks, train_segment_ids, train_labels 
) = bk.convert_examples_to_features(tokenizer, train_examples, max_seq_length=max_seq_length)
(test_input_ids, test_input_masks, test_segment_ids, test_labels
) = bk.convert_examples_to_features(tokenizer, test_examples, max_seq_length=max_seq_length)

In [ ]:
def build_model(max_seq_length): 
    in_id = tf.keras.layers.Input(shape=(max_seq_length,), name="input_ids")
    in_mask = tf.keras.layers.Input(shape=(max_seq_length,), name="input_masks")
    in_segment = tf.keras.layers.Input(shape=(max_seq_length,), name="segment_ids")
    bert_inputs = [in_id, in_mask, in_segment]
    
    bert_output = BertLayer(n_fine_tune_layers=2, pooling="first")(bert_inputs)
    dense = tf.keras.layers.Dense(256, activation='relu')(bert_output)
    pred = tf.keras.layers.Dense(1, activation='sigmoid')(dense)
    
    model = tf.keras.models.Model(inputs=bert_inputs, outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model

In [ ]:
model = build_model(max_seq_length)

# Instantiate variables
bk.initialize_vars(sess)

model.fit(
    [train_input_ids, train_input_masks, train_segment_ids], 
    train_labels,
    validation_data=([test_input_ids, test_input_masks, test_segment_ids], test_labels),
    epochs=1,
    batch_size=32
)